# oak-d × spectacularAI × nerfstudio Google Colab ワークフロー

このノートブックは、oak-dで撮影したデータ（spectacularAI形式）をGoogle Colabにアップロードし、nerfstudioで学習・可視化・ダウンロードするまでの一連の手順をまとめたものです。

---

## 概要
1. Colab環境セットアップ
2. データアップロード
3. nerfstudioインストール
4. spectacularAIデータをnerfstudio用に変換（必要なら）
5. nerfstudioで学習
6. 結果ダウンロード


In [ ]:
# 1. Colab環境セットアップ
!python --version
!nvidia-smi  # GPU確認
!pip install --upgrade pip

# GPU対応PyTorchのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# GPU利用可能性をチェック
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# 2. データをGoogle Colabにアップロード
# processed/ フォルダをzipファイルにしてアップロードしてください
# ローカルで: processed フォルダを右クリック → 圧縮 → processed.zip

from google.colab import files
import zipfile
import os

# zipファイルをアップロード
print("processed.zip ファイルをアップロードしてください（NerfStudio形式のデータ）")
uploaded = files.upload()

# zipファイルを展開
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"{filename} を展開しました")
        os.remove(filename)  # zipファイルを削除

# アップロードされたディレクトリ構造を確認
!ls -la

In [ ]:
# nerfstudioのインストール
!pip install nerfstudio

# nerfstudioの動作確認
import nerfstudio
print("✓ nerfstudio が正常にインポートできました")

# nerfstudioのバージョン確認（pipから）
!pip show nerfstudio | grep Version

# nerfstudioのコマンドが使用可能か確認
!ns-train --help | head -5

In [ ]:
# 3. アップロードしたデータの確認
# processed.zipを展開済みなので、NerfStudio形式データを確認
print("アップロードされたデータ構造:")
!find processed -type f | head -20

# transforms.jsonの存在確認
import os
if os.path.exists('processed/transforms.json'):
    print("✓ transforms.json が見つかりました")
else:
    print("✗ transforms.json が見つかりません")

# 画像フォルダの確認
if os.path.exists('processed/images'):
    image_count = len([f for f in os.listdir('processed/images') if f.endswith('.jpg')])
    print(f"✓ 画像フォルダに {image_count} 枚の画像があります")
else:
    print("✗ images フォルダが見つかりません")

In [ ]:
# 4. nerfstudioで学習
# processed/ フォルダのデータを使って学習を実行
!ns-train nerfacto --data processed/ --output-dir outputs/

print("学習コマンドが完了しました。")

In [ ]:
# 4-2. 学習状況の確認
import os
import glob

print("=== 学習状況確認 ===")

# outputs/フォルダの存在確認
if os.path.exists('outputs/'):
    print("✓ outputs/ フォルダが作成されました")
    
    # ディレクトリ構造表示
    print("\n=== outputs/ ディレクトリ構造 ===")
    !find outputs -type f | head -20
    
    # チェックポイントファイルの確認
    print("\n=== チェックポイントファイル(.ckpt) ===")
    ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
    if ckpt_files:
        for file in ckpt_files:
            print(f"✓ {file}")
        print(f"\n合計 {len(ckpt_files)} 個のチェックポイントファイルが見つかりました")
    else:
        print("⚠️ チェックポイントファイルがまだ見つかりません")
    
    # ログファイルの確認
    print("\n=== 最新ログ（最後の10行） ===")
    !find outputs -name "*log*" -type f -exec tail -5 {} \; 2>/dev/null || echo "ログファイルが見つかりません"
    
else:
    print("⚠️ outputs/ フォルダがまだ作成されていません")
    print("学習がまだ開始されていないか、エラーが発生した可能性があります")

print("\n=== GPU使用状況 ===")
!nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv,noheader,nounits

In [ ]:
# 5. 学習結果のダウンロード
# ※ files.download() はブラウザのダウンロードフォルダにファイルを保存します
from google.colab import files
import glob
import os

# outputs/ ディレクトリ内の全体構造を確認
print("学習結果のディレクトリ構造:")
!find outputs -type f | head -30

# すべての.ckptファイルを検索してダウンロード
print("\n学習済みモデル(.ckpt)をダウンロード中...")
ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
if ckpt_files:
    for file in ckpt_files:
        if os.path.isfile(file):
            print(f"ダウンロード中: {file}")
            files.download(file)
else:
    print("⚠️ .ckptファイルが見つかりませんでした")

# 設定ファイルをダウンロード
print("\n設定ファイル(config.yml)をダウンロード中...")
config_files = glob.glob('outputs/**/config.yml', recursive=True)
if config_files:
    for file in config_files:
        if os.path.isfile(file):
            print(f"ダウンロード中: {file}")
            files.download(file)
else:
    print("⚠️ config.ymlファイルが見つかりませんでした")

# その他の重要なファイルもダウンロード
print("\nその他のファイル(.json, .pth)をダウンロード中...")
other_files = glob.glob('outputs/**/*.json', recursive=True) + glob.glob('outputs/**/*.pth', recursive=True)
for file in other_files:
    if os.path.isfile(file):
        print(f"ダウンロード中: {file}")
        files.download(file)

print("\nダウンロード完了！ブラウザのダウンロードフォルダを確認してください。")